In [1]:
import xarray as xr
import numpy  as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import skill_metrics as sm

尝试用python统计极端事件

In [2]:
# year = ["2003","2004","2005","2006","2007","2008","2009"]
year = ["2009"]
months = ["4","5","6","7","8"]
ordata = {} # 当年的统计量
for year_ind in year:
    # xarray 读取数据
    diri = "/m2data2/yycheng/yycheng/MPAS/92-25km_VR_"+year_ind+"/postprocess/temp_data/"
    fili = year_ind+"_daily_precip_combine_CMORPH_MPAS.nc"
    ds_disk = xr.open_dataset(diri + fili)
    precip_obs = ds_disk["precip_CMORPH"]
    precip_mod = ds_disk["precip_MPAS"]
    ordata[year_ind] = {'obs':precip_obs,'mod':precip_mod}
    # 计算逐年05-08序列
    # year_obs_mean = precip_obs.mean(dim=["time"])
    # year_mod_mean = precip_mod.mean(dim=["Time"])
    # year_stats[year_ind] = sm.taylor_statistics( np.array(year_obs_mean).ravel() , np.array(year_mod_mean).ravel() )
    # 拆分逐月，计算统计量
    # month_stats = {} # 逐月的统计量
        #  taylor_stats1.append( sm.taylor_statistics(np.array(obs_mean),np.array(mod_mean)) )
        # months_obs_mean = year_obs_mean.sel(time = slice(year_ind+"-"+month_ind))
        # months_mod_mean = year_mod_mean.sel(Time = slice(year_ind+"-"+month_ind)) # MPAS中奇怪的时间坐标名称Time
        # month_stats[month_ind] = sm.taylor_statistics(np.array(months_obs_mean),np.array(months_mod_mean))
    # 逐月合并进逐年
    # year_stats[year_ind] = sm.taylor_statistics


In [3]:
obs_exterme = ordata[year[0]]['obs'][0,:,:].copy(deep = True)
mod_exterme = ordata[year[0]]['mod'][0,:,:].copy(deep = True)
obs_exterme[:,:] = 0.0
mod_exterme[:,:] = 0.0
threhold = 95
for i in range(0,ordata[year[0]]['obs'].shape[1]):
    for j in range(0,ordata[year[0]]['obs'].shape[2]):
        obs_exterme[i,j] = np.percentile(ordata[year[0]]['obs'][:,i,j], threhold)
        mod_exterme[i,j] = np.percentile(ordata[year[0]]['mod'][:,i,j], threhold)

merge to netcdf and output

In [4]:
exterme_output = xr.merge([obs_exterme, mod_exterme])
exterme_output

<xarray.Dataset>
Dimensions:        (lat: 160, latitude: 160, lon: 320, longitude: 320)
Coordinates:
    time           datetime64[ns] 2009-04-01
  * lat            (lat) float32 15.125 15.375 15.625 ... 54.375 54.625 54.875
  * lon            (lon) float32 60.125 60.375 60.625 ... 139.625 139.875
    Time           datetime64[ns] 2009-04-01
  * latitude       (latitude) float32 15.124999 15.374997 ... 54.874992
  * longitude      (longitude) float32 60.124992 60.374996 ... 139.87498
Data variables:
    precip_CMORPH  (lat, lon) float32 0.5446 0.66400003 ... 6.6438003 7.7274003
    precip_MPAS    (latitude, longitude) float64 3.149 3.29 ... 9.992 9.056

In [5]:
exterme_output.to_netcdf(path = diri + year[0] + "_exterme_precip.nc",mode = 'w')